#Miljødataanalyse - Kode Del

In [26]:
import pandas as pd

# Read the CSV file
weather_data = pd.read_csv("./weather_data.csv")

# View the first 5 rows
weather_data.head()


,time,temperature,humidity,wind_speed,cloud_cover,pressure
0,2025-04-16T09:00:00Z,13.2,66.7,3.3,91.1,1007.9
1,2025-04-16T10:00:00Z,14.9,64.8,3.2,89.4,1007.4
2,2025-04-16T11:00:00Z,16.5,62.3,3.6,91.5,1006.5
3,2025-04-16T12:00:00Z,17.6,60.9,4.4,91.3,1005.9
4,2025-04-16T13:00:00Z,17.9,59.8,5.5,98.7,1005.5


In [16]:
import requests
import pandas as pd
from pandasql import sqldf 
"""
Jeg måtte kjøre pip install pandasql i en egen notebook-celle
 for å få det til å funke, det gikk ikke å bare installere i terminal. 
 Jeg slettet den etterpå, men om det ikke funker for dere kan dere prøve det.
"""

'\nJeg måtte kjøre pip install pandasql i en egen notebook-celle\n for å få det til å funke, det gikk ikke å bare installere i terminal. \n Jeg slettet den etterpå, men om det ikke funker for dere kan dere prøve det.\n'

In [ ]:
"""Mitt forsøk på oppgave 2"""
"""
Hentet fra: https://api.met.no/ mer spesifikt hentet jeg API-en fra:
 https://api.met.no/weatherapi/locationforecast/2.0/documentation
"""


url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
# Denne API krever user agent, står på nettsiden
headers = {
    "User-Agent": "Gruppe43"
    "Id-Modified_Since": "2024-04-21"
    #Oppdater ID-midified since hver dag
}

params = {
    "lat": 63.4305,
    "lon": 10.3951
}

res = requests.get(url, headers=headers, params=params)
data = res.json()

timeseries = data["properties"]["timeseries"] #Gir en liste av værdata per time 

weather_data = [
    {
        "time": entry["time"],
        "temperature": entry["data"]["instant"]["details"].get("air_temperature"),
        "humidity": entry["data"]["instant"]["details"].get("relative_humidity"),
        "wind_speed": entry["data"]["instant"]["details"].get("wind_speed"),
        "cloud_cover": entry["data"]["instant"]["details"].get("cloud_area_fraction"),
        "pressure": entry["data"]["instant"]["details"].get("air_pressure_at_sea_level"),
    }
    for entry in timeseries  # loop gjennom værmeldingen for hver time
]

df = pd.DataFrame(weather_data) # Gjør om til pandas dataframe (Gjør det til en tabell)

#Bruke pandasql for å finne 10 varmeste tidene (Den sto i oppgaven vi måtte bruke pandasql så jeg bare la noe til, vi kan endre på det senere)
query = "SELECT time, temperature FROM df WHERE temperature IS NOT NULL ORDER BY temperature DESC LIMIT 10"
result = sqldf(query)
print(result)



                   time  temperature
0  2025-04-16T13:00:00Z         17.9
1  2025-04-16T12:00:00Z         17.6
2  2025-04-16T14:00:00Z         16.8
3  2025-04-16T11:00:00Z         16.5
4  2025-04-16T10:00:00Z         14.9
5  2025-04-16T15:00:00Z         14.7
6  2025-04-16T17:00:00Z         14.0
7  2025-04-17T14:00:00Z         13.6
8  2025-04-16T16:00:00Z         13.5
9  2025-04-16T18:00:00Z         13.5


In [31]:
"""Mitt forsøk på oppgave 3"""

def vis_manglende_verdier(df):
    """Finner og viser manglende verdier"""
    print("Manglende verdier per kolonne:")
    print(df.isnull().sum()) #Viser antall manglende verdier per kolonne (Det er ingen mangler :( vi må lage mangler, men tror det skal gå fint

def rens_data(df):
    """Renser manglende og 'rare' verdier"""
    #Lager en kopi så vi ikke endrer originalen
    df = df.copy()

    #Dette er eksempler på data rensing som jeg fant på nett vi kan finpusse sammen

    #Fjerner rader der temperaturen mangler
    df = df.dropna(subset=["temperature"]) 

    #Fyller NaN temperatur verdier med medianen
    df["temperature"] = df["temperature"].fillna(df["temperature"].mean())

    #Gjør det samme men med fuktighetsverdier
    df["humidity"] = df["humidity"].fillna(df["humidity"].mean())

    #Fjerner fuktighets verdier som ikke går an (over 100 eller under 0)
    df = df[(df["humidity"] >= 0) & (df["humidity"] <= 100)]

    #Erstatt ekstreme temperaturer som ikke er mulig/ gir mening med NaN
    df.loc[df["temperature"] < -60, "temperature"] = pd.NA
    df.loc[df["temperature"] > 60, "temperature"] = pd.NA

    return df

def formater_data(df):
    """Formaterer dataene for å gjøre dem mer strukturert og leselig"""
    df = df.copy()

    #Gjør tid kolonnen om til datetime
    df["time"] = pd.to_datetime(df["time"])

    #Runder verdier for lesbarhet
    df["temperature"] = df["temperature"].round(1)
    df["humidity"] = df["humidity"].round(0)
    df["wind_speed"] = df["wind_speed"].round(1)

    #Gir nye navn til kolonner om vi vil
    df.rename(columns={"wind_speed": "wind_speed_mps"}, inplace=True)

    #Endre rekkefølgen på kolonnene
    kolonne_rekkefølge = ["time", "temperature", "humidity", "wind_speed_mps", "cloud_cover", "pressure"]
    df = df[[col for col in kolonne_rekkefølge if col in df.columns]]

    return df

def flagg_kuldegrader(df):
    """Legger til en ny kolonne for kuldegrader"""
    df = df.copy()
    df["Kuldegrader"] = [t < 0 if pd.notnull(t) else False for t in df["temperature"]]
    return df

def prepare_data(df):
    """Kombinerer alt"""
    vis_manglende_verdier(df)
    df = rens_data(df)
    df = formater_data(df)
    df = flagg_kuldegrader(df)
    return df

df_cleaned = prepare_data(df)

query = "SELECT time, temperature FROM df_cleaned WHERE temperature IS NOT NULL ORDER BY time DESC LIMIT 10"
result = sqldf(query)
print(result)

Manglende verdier per kolonne:
time           0
temperature    0
humidity       0
wind_speed     0
cloud_cover    0
pressure       0
dtype: int64
                         time  temperature
0  2025-04-26 06:00:00.000000          5.7
1  2025-04-26 00:00:00.000000          5.7
2  2025-04-25 18:00:00.000000          9.4
3  2025-04-25 12:00:00.000000         10.3
4  2025-04-25 06:00:00.000000          4.7
5  2025-04-25 00:00:00.000000          5.8
6  2025-04-24 18:00:00.000000         10.3
7  2025-04-24 12:00:00.000000          9.3
8  2025-04-24 06:00:00.000000          5.2
9  2025-04-24 00:00:00.000000          5.8


In [ ]:

formatert_df = formater_data(df)

data = pd.DataFrame(formatert_df).to_csv("weather_data.csv", index=False)
#Lag ny csv fil under data
#Lag en for loop som sjekker om infoen i filen eksisterer, hvis ikke, append

# Read the CSV file
weather_data = pd.read_csv("./weather_data.csv")

# View the first 5 rows
weather_data.head(40)

,time,temperature,humidity,wind_speed_mps,cloud_cover,pressure
0,2025-04-16 09:00:00+00:00,13.2,67.0,3.3,91.1,1007.9
1,2025-04-16 10:00:00+00:00,14.9,65.0,3.2,89.4,1007.4
2,2025-04-16 11:00:00+00:00,16.5,62.0,3.6,91.5,1006.5
3,2025-04-16 12:00:00+00:00,17.6,61.0,4.4,91.3,1005.9
4,2025-04-16 13:00:00+00:00,17.9,60.0,5.5,98.7,1005.5
5,2025-04-16 14:00:00+00:00,16.8,61.0,6.4,100.0,1005.6
6,2025-04-16 15:00:00+00:00,14.7,77.0,5.0,100.0,1006.1
7,2025-04-16 16:00:00+00:00,13.5,85.0,4.7,100.0,1006.1
8,2025-04-16 17:00:00+00:00,14.0,80.0,4.6,100.0,1006.1
9,2025-04-16 18:00:00+00:00,13.5,77.0,3.0,73.8,1006.2
